In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
from ag_ep import *
from local_enc import *
from global_oneshot import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
## Load dataset
data_path = "../gmm_10size"
Data = []
for i in range(10):
    data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % ((i+1)*10))).float()
    Data.append(data)

# NUM_DATASETS, N, D = Data[0].shape
K = 3 ## number of clusters
D = 2
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE =  1e-4
CUDA = torch.cuda.is_available()

DEVICE = torch.device('cuda:1')
Model_Params = (K, D, MCMC_SIZE)
PATH = 'ag-10size-%dSample' % SAMPLE_SIZE

In [3]:
Train_Params = (NUM_EPOCHS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
train_10size(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

100
epoch=0\500, data order=tensor([5, 7, 2, 6, 1, 3, 8, 4, 9, 0])


In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)